In [35]:
#Install cell

!pip install pydriller
!pip install bokeh
!pip install tqdm
!pip install pathlib
!pip install pandas
!pip install numpy
!pip install matplotlib


In [36]:
# Importing Required Libraries

import pydriller 
from pydriller import ModificationType
import pandas as pd
import subprocess
from pathlib import Path
from git import Repo
from datetime import datetime
import glob

In [37]:
#  Import only from the App folder for a current tag 
current_tag = 'v3.5.0'

gr = pydriller.Git(path_to_repo)
gr.checkout(current_tag)

entities_paths = glob.glob(path_to_repo + '/**/*.rb', recursive=True)

entities_paths__rb = [p for p in entities_paths if p.endswith(file_type)]


#remove initial path
entities_paths = [p[11:] for p in entities_paths]
entities_paths = [p for p in entities_paths if p.startswith('app')]

print('Total number of files in /app: ', len(entities_paths))


print('Total number of ruby files in entire repo: ', len(entities_paths__rb))


Total number of files in /app:  809
Total number of ruby files in entire repo:  1793


In [38]:

# Path to the repository
repo_path = Path('./mastodon')
path_to_repo = './mastodon'

# Check if the repository exists
if repo_path.exists():
    print('Repository exists!')
else:
    # Clone the repository
    subprocess.call(['git', 'clone', 'git://github.com/tootsuite/mastodon.git'])


index_name = "mastodon"
start_tag =  'v3.0.0'
current_tag = 'v3.5.0'
file_type = '.rb'

from_date = datetime(2020, 1, 1)

to_date =  datetime(2022, 6, 1)

# Tags
repo = pydriller.Repository('./mastodon', from_tag=start_tag, to_tag=current_tag)
# Dates
# repo = pydriller.Repository('./mastodon', since=from_date, to=to_date)



Repository exists!


In [39]:
#  Get number of commits & modified files
commits = 0
modified_files = 0
modified__ruby_files = 0

for commit in repo.traverse_commits():
    commits += 1
    modified_files += len(commit.modified_files)
    for file in commit.modified_files:
        if file.filename.endswith(file_type):
            modified__ruby_files += 1

print('Total Number of commits: ', commits)
print('Total Number of modified files: ', modified_files)
print('Total Number of modified ruby files: ', modified__ruby_files)

Total Number of commits:  3428
Total Number of modified files:  16291
Total Number of modified ruby files:  3475


In [40]:
# Complexity of a file:

file_name = 'app/helpers/settings_helper.rb'

# Get the complexity of a file
for commit in repo.traverse_commits():
    for modified_file in commit.modified_files:
        
        if modified_file.new_path == file_name:
            print(modified_file.complexity)

8
8
8
10
10
10
10
10
10
9
9


In [41]:
def count_loc(file_path):
    loc = 0
    try:
        with open(path_to_repo + '/' + file_path) as f:
            for line in f:
                if not line.isspace():
                    loc += 1
    except FileNotFoundError:
        loc = 0
    return loc

In [42]:
metrics = dict()
loc = 'loc'
changes = 'changes'

for entity_path in entities_paths:
    metrics[entity_path] = {loc : count_loc(entity_path), changes : 0}


for commit in repo.traverse_commits():
    #print(commit)
    for m_file in commit.modified_files:
        if m_file.old_path and (m_file.new_path != m_file.old_path):
            print(m_file.old_path, "->", m_file.new_path)
        if m_file.new_path in entities_paths:
            metrics[m_file.new_path][changes] += 1

csv_like = [('fullpath',loc,changes)]

for fullpath,values in metrics.items():
    csv_like.append((fullpath,values[loc],values[changes]))

app/javascript/mastodon/components/extended_video_player.js -> None
config/webpack/test.js -> config/webpack/tests.js
app/services/fetch_remote_account_service.rb -> None
spec/services/fetch_remote_account_service_spec.rb -> None
app/controllers/concerns/obfuscate_filename.rb -> None
spec/controllers/concerns/obfuscate_filename_spec.rb -> None
app/controllers/admin/followers_controller.rb -> None
app/views/admin/followers/index.html.haml -> None
app/middleware/handle_bad_encoding_middleware.rb -> None
spec/middleware/handle_bad_encoding_middleware_spec.rb -> None
app/workers/after_remote_follow_request_worker.rb -> None
app/workers/after_remote_follow_worker.rb -> None
app/workers/notification_worker.rb -> None
app/workers/processing_worker.rb -> None
app/workers/pubsubhubbub/confirmation_worker.rb -> None
app/workers/pubsubhubbub/delivery_worker.rb -> None
app/workers/pubsubhubbub/distribution_worker.rb -> None
app/workers/pubsubhubbub/raw_distribution_worker.rb -> None
app/workers/pu

In [ ]:
metrics


{'app/services/notify_service.rb': {'loc': 125, 'changes': 11},
 'app/services/fan_out_on_write_service.rb': {'loc': 114, 'changes': 10},
 'app/services/favourite_service.rb': {'loc': 36, 'changes': 3},
 'app/services/post_status_service.rb': {'loc': 162, 'changes': 9},
 'app/services/vote_service.rb': {'loc': 66, 'changes': 1},
 'app/services/after_block_service.rb': {'loc': 24, 'changes': 2},
 'app/services/follow_service.rb': {'loc': 71, 'changes': 8},
 'app/services/block_service.rb': {'loc': 21, 'changes': 0},
 'app/services/fetch_remote_status_service.rb': {'loc': 12, 'changes': 2},
 'app/services/tag_search_service.rb': {'loc': 74, 'changes': 0},
 'app/services/report_service.rb': {'loc': 56, 'changes': 4},
 'app/services/block_domain_service.rb': {'loc': 39, 'changes': 4},
 'app/services/deliver_to_device_service.rb': {'loc': 66, 'changes': 1},
 'app/services/unsuspend_account_service.rb': {'loc': 79, 'changes': 10},
 'app/services/batched_remove_status_service.rb': {'loc': 72,

In [ ]:

csv_like

[('fullpath', 'loc', 'changes'),
 ('app/services/notify_service.rb', 125, 11),
 ('app/services/fan_out_on_write_service.rb', 114, 10),
 ('app/services/favourite_service.rb', 36, 3),
 ('app/services/post_status_service.rb', 162, 9),
 ('app/services/vote_service.rb', 66, 1),
 ('app/services/after_block_service.rb', 24, 2),
 ('app/services/follow_service.rb', 71, 8),
 ('app/services/block_service.rb', 21, 0),
 ('app/services/fetch_remote_status_service.rb', 12, 2),
 ('app/services/tag_search_service.rb', 74, 0),
 ('app/services/report_service.rb', 56, 4),
 ('app/services/block_domain_service.rb', 39, 4),
 ('app/services/deliver_to_device_service.rb', 66, 1),
 ('app/services/unsuspend_account_service.rb', 79, 10),
 ('app/services/batched_remove_status_service.rb', 72, 8),
 ('app/services/verify_link_service.rb', 36, 0),
 ('app/services/remove_from_followers_service.rb', 19, 1),
 ('app/services/remove_status_service.rb', 121, 11),
 ('app/services/suspend_account_service.rb', 82, 9),
 ('app/

In [ ]:
import pandas as pd

df_complexity = pd.DataFrame(csv_like[1:], columns=csv_like[0])
df_complexity

fullpath  loc  changes
0              app/services/notify_service.rb  125       11
1    app/services/fan_out_on_write_service.rb  114       10
2           app/services/favourite_service.rb   36        3
3         app/services/post_status_service.rb  162        9
4                app/services/vote_service.rb   66        1
..                                        ...  ...      ...
804    app/policies/account_warning_policy.rb   13        2
805           app/policies/ip_block_policy.rb   12        1
806       app/policies/announcement_policy.rb   15        1
807        app/policies/application_policy.rb   16        0
808              app/policies/relay_policy.rb    6        0

[809 rows x 3 columns]

In [ ]:
import plotly.express as px

fig = px.scatter(df_complexity, x="loc", y="changes",
                hover_data=['fullpath'])
fig.show()

In [ ]:
candidates = ['app/models/status.rb','app/models/account.rb','app/lib/feed_manager.rb',
              'app/models/user.rb','app/helpers/application_helper.rb','app/helpers/languages_helper.rb']

rows = []

for commit in repo.traverse_commits():
    for m_file in commit.modified_files:
        if m_file.new_path in candidates:
            d = {}
            d['fullpath'] = m_file.new_path
            d['date'] = commit.committer_date
            d[loc] = m_file.nloc
            rows.append(d)
df = pd.DataFrame(rows)

In [ ]:
fig = px.line(df, x='date', y='loc', color='fullpath')
fig.show()

In [ ]:
older_tag = 'v3.2.0'

repo = pydriller.Repository(path_to_repo, from_tag = older_tag, to_tag = current_tag)

rows = []

for commit in repo.traverse_commits():
    for m_file in commit.modified_files:
        if m_file.new_path in candidates:
            d = {}
            d['fullpath'] = m_file.new_path
            d['date'] = commit.committer_date
            d[loc] = m_file.nloc
            rows.append(d)
df = pd.DataFrame(rows)

fig = px.line(df, x='date', y='loc', color='fullpath')
fig.show()